In [0]:
!pip install sentencepiece
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchtext import datasets, data
from torch.utils.data import DataLoader
from torch.autograd import Variable
import sentencepiece as spm
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math, copy, time
%matplotlib inline
seaborn.set_context(context="talk")
!wget http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
!tar zxvf training-parallel-nc-v13.tgz -C data/
DEVICE = 'cuda'

--2020-02-22 13:17:17--  http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113157482 (108M) [application/x-gzip]
Saving to: ‘training-parallel-nc-v13.tgz.1’

training-parallel-n 100%[===================>] 107.92M  11.0MB/s    in 12s     

2020-02-22 13:17:30 (9.07 MB/s) - ‘training-parallel-nc-v13.tgz.1’ saved [113157482/113157482]

training-parallel-nc-v13/
training-parallel-nc-v13/news-commentary-v13.ru-en.ru
training-parallel-nc-v13/news-commentary-v13.cs-en.en
training-parallel-nc-v13/news-commentary-v13.de-en.de
training-parallel-nc-v13/news-commentary-v13.ru-en.en
training-parallel-nc-v13/news-commentary-v13.zh-en.zh
training-parallel-nc-v13/news-commentary-v13.zh-en.en
training-parallel-nc-v13/news-commentary-v13.cs-en.cs
training-parallel-nc-v13/news-commen

In [0]:
with open('data/news-commentary-v13.ru-en.en') as f:
    with open('data/text.en', 'w') as out:
            out.write(f.read().lower())
        
spm.SentencePieceTrainer.Train('--input=data/text.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [0]:
# tokenize russian
with open('data/news-commentary-v13.ru-en.ru') as f:
    with open('data/text.ru', 'w') as out:
            out.write(f.read().lower())
        
spm.SentencePieceTrainer.Train('--input=data/text.ru --model_prefix=bpe_ru --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [0]:
tok_ru = spm.SentencePieceProcessor()
tok_ru.load('bpe_ru.model')

tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

SRC = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_ru.encode_as_pieces(x),
    batch_first=True,
)

TGT = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),
    batch_first=True,
)

fields = (('src', SRC), ('tgt', TGT))

In [0]:
from tqdm import tqdm

In [0]:
with open('data/text.ru') as f:
    src_snt = list(map(str.strip, f.readlines()))
    
with open('data/text.en') as f:
    tgt_snt = list(map(str.strip, f.readlines()))
    
examples = [data.Example.fromlist(x, fields) for x in tqdm(zip(src_snt, tgt_snt))]
test = data.Dataset(examples[-1000:], fields)
train, valid = data.Dataset(examples[:-1000], fields).split(0.9)

235159it [01:15, 3107.65it/s]


In [0]:
print('src: ' + " ".join(train.examples[100].src))
print('tgt: ' + " ".join(train.examples[100].tgt))

src: ▁совсем ▁недавно ▁дональд ▁к ер р , ▁заместитель ▁главы ▁национальной ▁разведки ▁сша , ▁предупредил ▁о ▁том , ▁что ▁ « шпи оны ▁не ▁отвечают ▁за ▁большую ▁часть ▁утечки ▁ин ф ормации ▁и ▁секре тов ▁наших ▁правитель ственных ▁программ .... в ▁действ ительности ▁меня ▁больше ▁беспокоит ▁тот ▁ ф акт , ▁что ▁столь ▁многие ▁возможности , ▁от ▁которых ▁мы ▁все ▁будем ▁зависеть , ▁больше ▁не ▁разрабаты ваются ▁в ▁правитель ственных ▁лабораториях ▁по ▁правитель ственным ▁контрак там » .
tgt: ▁ j ust ▁recently , ▁donald ▁k err , ▁the ▁us ▁deputy ▁director ▁of ▁national ▁intelligence , ▁warned ▁that ▁ “ ma j or ▁losses ▁of ▁information ▁and ▁value ▁for ▁our ▁government ▁programs ▁typically ▁aren ’ t ▁from ▁spies .... in ▁fact , ▁one ▁of ▁the ▁great ▁concerns ▁i ▁have ▁is ▁that ▁so ▁much ▁of ▁the ▁new ▁capabilities ▁that ▁we ’ re ▁all ▁going ▁to ▁depend ▁on ▁aren ’ t ▁any ▁longer ▁developed ▁in ▁government ▁labs ▁under ▁government ▁contract . ”


In [0]:
len(train), len(valid), len(test)

(210743, 23416, 1000)

In [0]:
TGT.build_vocab(train, min_freq=5)
SRC.build_vocab(train, min_freq=5)

In [0]:
from transformer import make_model, Batch

    
class BucketIteratorWrapper(DataLoader):
    __initialized = False

    def __init__(self, iterator: data.Iterator):
#         super(BucketIteratorWrapper,self).__init__()
        self.batch_size = iterator.batch_size
        self.num_workers = 1
        self.collate_fn = None
        self.pin_memory = False
        self.drop_last = False
        self.timeout = 0
        self.worker_init_fn = None
        self.sampler = iterator
        self.batch_sampler = iterator
        self.__initialized = True

    def __iter__(self):
        return map(
            lambda batch: Batch(batch.src, batch.tgt, pad=TGT.vocab.stoi['<pad>']),
            self.batch_sampler.__iter__()
        )

    def __len__(self):
        return len(self.batch_sampler)
    
class MyCriterion(nn.Module):
    def __init__(self, pad_idx):
        super(MyCriterion, self).__init__()
        self.pad_idx = pad_idx
        self.criterion = nn.CrossEntropyLoss(reduction='sum', ignore_index=pad_idx)
        
    def forward(self, x, target):
        x = x.contiguous().permute(0,2,1)
        ntokens = (target != self.pad_idx).data.sum()
        
        return self.criterion(x, target) / ntokens

In [0]:
torch.cuda.empty_cache()

batch_size = 64
num_epochs = 20

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train, valid, test), 
                                              batch_sizes=(batch_size, batch_size, batch_size), 
                                  sort_key=lambda x: len(x.src),
                                  shuffle=True,
                                  device=DEVICE,
                                  sort_within_batch=False)
                                  
train_iter = BucketIteratorWrapper(train_iter)
valid_iter = BucketIteratorWrapper(valid_iter)
test_iter = BucketIteratorWrapper(test_iter)

In [0]:
model = make_model(len(SRC.vocab), len(TGT.vocab), N=6, d_model=512, d_ff=2048, h=8, dropout=0.1)
model = model.to(DEVICE)

In [0]:
pad_idx = TGT.vocab.stoi["<pad>"]
criterion = MyCriterion(pad_idx)

In [0]:
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

In [0]:
model.eval()
for batch in train_iter:
    print(batch.src.shape, batch.tgt_y.shape)
    out = model.forward(batch)
    print(out.shape)
    out = F.log_softmax(out, dim=-1)
    print(out.shape)
    break

torch.Size([64, 50]) torch.Size([64, 49])
torch.Size([64, 49, 28298])
torch.Size([64, 49, 28298])


In [0]:
def train_epoch(data_iter, model, criterion, optimizer):
    total_loss = 0
    data_iter = tqdm(data_iter, position=0, leave=True)
    counter = 0
    for batch in data_iter:
        optimizer.zero_grad()
        out = model.forward(batch)
        out = F.log_softmax(out, dim=-1)
        loss = criterion(out, batch.tgt_y)
        loss.backward()
        optimizer.step()
        total_loss += loss
        data_iter.set_postfix(loss = loss)
        counter +=1
    total_loss /= counter
    return total_loss

def valid_epoch(data_iter, model, criterion):
    total_loss = 0
    data_iter = tqdm(data_iter)
    counter = 0
    for batch in data_iter:
        
        out = model.forward(batch)
        out = F.log_softmax(out, dim=-1)
        loss = criterion(out, batch.tgt_y)
        
        total_loss += loss
        data_iter.set_postfix(loss = loss)
        counter +=1
        
    total_loss /= counter
    return total_loss

In [0]:
num_epochs = 1 # try 20
for epoch in range(num_epochs):
    model.train()
    loss = train_epoch(train_iter, model, criterion, optimizer)
    print('train', loss)
    
    model.eval()
    with torch.no_grad():
        loss = valid_epoch(valid_iter, model, criterion)
        scheduler.step(loss)
        print('valid', loss)

  0%|          | 1/366 [00:00<00:45,  8.06it/s, loss=tensor(5.2908, device='cuda:0')]

train tensor(5.4829, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 366/366 [00:36<00:00,  9.95it/s, loss=tensor(5.7329, device='cuda:0')]


valid tensor(5.1875, device='cuda:0')


In [0]:
from math import log
from numpy import array
from numpy import argmax
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0
def greedy_search(model, src, src_mask, max_len=10):
    start_symbol = TGT.vocab.stoi["<s>"]
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(Variable(ys), 
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)), memory, src_mask)
        prob = F.log_softmax(out, dim=-1)
        _, next_word = torch.max(prob[:, -1], dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [0]:
def beam_search(model, src, src_mask, max_len=10, k=5):
    start_symbol = TGT.vocab.stoi["<s>"]
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    candidate_sequences = [(ys, 0)]
    final_candidates = []
    for t in range(max_len):
        new_candidate_sequences = []
        for i, current_candidate in enumerate(candidate_sequences):
            out = model.decode(Variable(current_candidate[0]), 
                                Variable(subsequent_mask(current_candidate[0].size(1)).type_as(src.data)), memory, src_mask)
            prob = F.log_softmax(out, dim=-1)
            prob, next_words = torch.topk(prob[:, -1], dim = 1, k = prob.shape[-1], sorted=False)
            for j, next_word in enumerate(next_words[0]):
                next_word = next_word.item()
                new_candidate_sequence = torch.cat([current_candidate[0], torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
                new_candidate_prob = current_candidate[1] + prob[0][j].item()
                new_candidate_sequences.append((new_candidate_sequence, new_candidate_prob))
            new_candidate_sequences = sorted(new_candidate_sequences, key=lambda x: x[1], reverse=True)[:k]
        new_candidate_sequences_ = []
        for (seq, prob) in new_candidate_sequences:
            if seq[0][-1] == TGT.vocab.stoi['</s>']:
                final_candidates.append((seq, prob))
                k = k - 1
            else:
                new_candidate_sequences_.append((seq, prob))
        candidate_sequences = new_candidate_sequences_
        if k <= 0:
            break
    final_candidates.extend(candidate_sequences)
    return sorted(final_candidates, key=lambda x: x[1], reverse=True)[0]

In [0]:
greedy_search(model, src, src_key_padding_mask)

tensor([[  2,   4,   9,  33, 410,   3,   3,   3,   3,   3]], device='cuda:0')

In [0]:
beam_search(model, src, src_key_padding_mask, k = 1)

(tensor([[  2,   4,   9,  33, 410,   3]], device='cuda:0'), -8.522655010223389)

In [0]:
beam_search(model, src, src_key_padding_mask, k = 2)

(tensor([[ 2, 30,  4, 47,  3]], device='cuda:0'), -9.36168098449707)

In [0]:
model.eval()
with torch.no_grad():
    for i, batch in enumerate(valid_iter):
        src = batch.src[:1]
        src_key_padding_mask = src != SRC.vocab.stoi["<pad>"]
        beam = beam_search(model, src, src_key_padding_mask)
        
        seq = []
        for i in range(1, src.size(1)):
            sym = SRC.vocab.itos[src[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_ru.decode_pieces(seq)
        print("\nSource:", seq)
        
        print("Translation:")
        for pred, pred_proba in beam:                
            seq = []
            for i in range(1, pred.size(1)):
                sym = TGT.vocab.itos[pred[0, i]]
                if sym == "</s>": break
                seq.append(sym)
            seq = tok_en.decode_pieces(seq)
            print(f"pred {pred_proba:.2f}:", seq)
                
        seq = []
        for i in range(1, batch.tgt.size(1)):
            sym = TGT.vocab.itos[batch.tgt[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        print("Target:", seq)
        break


Source: сосуществование
Translation:


ValueError: ignored

In [0]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

In [0]:
hypotheses = []
references = []
verbose = 0
model.eval()
with torch.no_grad():
    for batch in test_iter:
        model.eval()
        src = batch.src[:1]
        src_key_padding_mask = src != SRC.vocab.stoi["<pad>"]
        pred = greedy_search(model, src, src_key_padding_mask)
        
        seq = []
        for i in range(1, src.size(1)):
            sym = SRC.vocab.itos[src[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_ru.decode_pieces(seq)
        if verbose: print("\nSource:", seq)

        seq = []
        for i in range(1, pred.size(1)):
            sym = TGT.vocab.itos[pred[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        if verbose: print("\nTranslation:", seq)
        hypotheses.append(seq)
                
        seq = []
        for i in range(1, batch.tgt.size(1)):
            sym = TGT.vocab.itos[batch.tgt[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        if verbose: print("Target:", seq)
        references.append(seq)

In [0]:
corpus_bleu(references, hypotheses, 
            smoothing_function=SmoothingFunction().method3,
            auto_reweigh=True
           )

0.0034547118824337967

In [0]:
hypotheses = []
references = []
verbose = 0
model.eval()
with torch.no_grad():
    for batch in test_iter:
        model.eval()
        src = batch.src[:1]
        src_key_padding_mask = src != SRC.vocab.stoi["<pad>"]
        pred, proba = beam_search(model, src, src_key_padding_mask)
        
        seq = []
        for i in range(1, src.size(1)):
            sym = SRC.vocab.itos[src[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_ru.decode_pieces(seq)
        if verbose: print("\nSource:", seq)

        seq = []
        for i in range(1, pred.size(1)):
            sym = TGT.vocab.itos[pred[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        if verbose: print("\nTranslation {} prob {}:".format(seq, proba))
        hypotheses.append(seq)
                
        seq = []
        for i in range(1, batch.tgt.size(1)):
            sym = TGT.vocab.itos[batch.tgt[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        if verbose: print("Target:", seq)
        references.append(seq)

In [0]:
corpus_bleu(references, hypotheses, 
            smoothing_function=SmoothingFunction().method3,
            auto_reweigh=True
           )

0.007427678827096992

<div id="disqus_thread"></div>
<script>
    /**
     *  RECOMMENDED CONFIGURATION VARIABLES: EDIT AND UNCOMMENT THE SECTION BELOW TO INSERT DYNAMIC VALUES FROM YOUR PLATFORM OR CMS.
     *  LEARN WHY DEFINING THESE VARIABLES IS IMPORTANT: https://disqus.com/admin/universalcode/#configuration-variables
     */
    /*
    var disqus_config = function () {
        this.page.url = PAGE_URL;  // Replace PAGE_URL with your page's canonical URL variable
        this.page.identifier = PAGE_IDENTIFIER; // Replace PAGE_IDENTIFIER with your page's unique identifier variable
    };
    */
    (function() {  // REQUIRED CONFIGURATION VARIABLE: EDIT THE SHORTNAME BELOW
        var d = document, s = d.createElement('script');
        
        s.src = 'https://EXAMPLE.disqus.com/embed.js';  // IMPORTANT: Replace EXAMPLE with your forum shortname!
        
        s.setAttribute('data-timestamp', +new Date());
        (d.head || d.body).appendChild(s);
    })();
</script>
<noscript>Please enable JavaScript to view the <a href="https://disqus.com/?ref_noscript" rel="nofollow">comments powered by Disqus.</a></noscript>